In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from problems import *

In [ ]:
Problem().name
QUBOBase().name
Distribution(loc=0, scale=1).name
QUBO().name

In [ ]:
import os
from functools import partial
from torch.utils.data import DataLoader

In [ ]:
from scripts.create_problem import load_problem_set
import problems as pbs
from dpt.data import *
from train_dpt import *

In [ ]:
problems = load_problem_set('data/normal_2/Normal(0, 1)/test')
for problem in problems:
    print(problem.name)

In [ ]:
# trajectories = defaultdict(int)
# results_dir = 'results/normal'
# output_dir = 'trajectories/normal'

# for suffix in ('train', 'val', 'test'):
#     for problem_class in tqdm(os.listdir(results_dir)):
#         problem_path = os.path.join(results_dir, problem_class, suffix)
#         solvers = [solver[:-5] for solver in os.listdir(problem_path)]
#         for solver in tqdm(solvers):
#             solver_path = os.path.join(problem_path, solver)
#             results = load_results(solver_path)
#             for problem, v in tqdm(results.items()):
#                 x = torch.tensor(v['x_list']).float()
#                 y = torch.tensor(v['y_list']).float()
#                 states = torch.cat([x, y.unsqueeze(-1)], dim=-1)
#                 out_path = os.path.join(output_dir, suffix, problem)
#                 os.makedirs(out_path, exist_ok=True)
#                 for seed in range(len(states)):
#                     torch.save(states[seed], os.path.join(out_path, f'{trajectories[problem]:08d}.pt'))
#                     trajectories[problem] += 1

In [ ]:
results_dir = 'results/normal'
dataset = OfflineDataset(problems, seq_len=10, results_dir=results_dir, suffix='test', ad_ratio=1.0, action='point', target_action='gt')
sample = dataset[0]

for k, v in sample.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

# print(sample['query_state'])
# print(sample['target_action'])
# print(sample['target_state'])

In [ ]:
collate_fn = partial(custom_collate_fn, problem_classes=[getattr(pbs, 'Distribution')])
dataloader = DataLoader(dataset=dataset, batch_size=10, collate_fn=collate_fn)
batch = next(iter(dataloader))

for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
# from train_dpt import ProblemDataModule, DPTSolver, L
from utils import load_config

In [ ]:
config = load_config('configs/config.yaml')
config['batch_size'] = 3

config['action'] = "point"
# target_action: "gt"
# ad_ratio: 0.0
config['model_params']['action_dim'] = 50

datamodule = ProblemDataModule(config)
datamodule.setup()

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

In [ ]:
# for k, v in batch.items():
#     if isinstance(v, torch.Tensor):
#         print(k, v.dtype, v.shape)
#     else:
#         print(k, type(v))

In [ ]:
model = DPTSolver(config)
model.configure_model()

In [ ]:
rewards = model.reward_model.offline(
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"]
)
rewards.shape

In [ ]:
outputs = model.model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=rewards,
)
outputs.shape

In [ ]:
trainer = L.Trainer(
    logger=None,
    precision=config["precision"] if config["strategy"] != "fsdp" else None,
    max_epochs=config["max_epochs"],
    log_every_n_steps=config["log_every_n_steps"],
    default_root_dir=config["wandb_params"]["save_dir"],
    enable_model_summary=True,
    use_distributed_sampler=False,
    # strategy=config["strategy"]
)